In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
import seaborn as sns

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Aioros

In [4]:
df = pd.read_pickle(f'../workdata/miro/part_0.pkl')
df = df.groupby(['diag', 'db']).head(5)
df.groupby(['diag', 'db']).size()

diag  db                  
AF    WFDB_CPSC2018           5
      WFDB_CPSC2018_2         5
      WFDB_ChapmanShaoxing    5
      WFDB_Ga                 5
      WFDB_PTB                1
      WFDB_PTBXL              5
SR    WFDB_CPSC2018           5
      WFDB_ChapmanShaoxing    5
      WFDB_Ga                 5
      WFDB_Ningbo             5
      WFDB_PTB                5
      WFDB_PTBXL              5
dtype: int64

### Aioros Large

In [12]:
df = pd.read_pickle(f'../workdata/miro/part_0.pkl')
df = df.append(pd.read_pickle(f'../workdata/miro/part_1.pkl'))
df = df.append(pd.read_pickle(f'../workdata/miro/part_2.pkl'))

df.groupby(['diag', 'db']).size()
print(df.shape)
df.head()

(9951, 7)


,db,filename,diag,total_diags,db_freq,time,data
0,WFDB_CPSC2018,A4644.mat,SR,1,500,10.0,"[[28.56657366217581, 193.9291784767534, 189.77..."
1,WFDB_PTBXL,HR15248.mat,SR,1,500,10.0,"[[6.123519477705961, 123.8742732531606, 185.69..."
2,WFDB_Ga,E02092.mat,SR,1,500,10.0,"[[-26.92540258258549, 1.9379950802880224, -49...."
3,WFDB_PTBXL,HR18712.mat,SR,5,500,10.0,"[[19.638712349506292, -30.40160146972147, -18...."
4,WFDB_PTBXL,HR21119.mat,AF,4,500,10.0,"[[-15.478587022808028, 37.04396020085395, 22.4..."


In [22]:
leads = ('I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6')
sns.set(rc={'figure.figsize':(11.7,8.27)})
df2 = pd.DataFrame(columns=['diag', 'data'])

PRESENTATION_MODE = False

if PRESENTATION_MODE:
    if not os.path.exists(f'../workdata/aioros/SR/'):
        os.makedirs(f'../workdata/aioros/SR/')
    if not os.path.exists(f'../workdata/aioros/AF/'):
        os.makedirs(f'../workdata/aioros/AF/')
        
rows = df.shape[0]

for i, row in df.iterrows():
    print(f'{i}/{rows}')
    try:
        ecg_signal = row['data'][-2] # V5
        _, rpeaks = nk.ecg_peaks(ecg_signal, sampling_rate=100)
        ecg_r_peaks = rpeaks['ECG_R_Peaks']
        ecg_r_peaks = list(map(lambda x: x - 20, ecg_r_peaks))
        ecg_size = len(ecg_signal)
        ecg_r_peaks.insert(0, 0)
        ecg_r_peaks.insert(ecg_size, ecg_size)

        n_peaks = len(ecg_r_peaks)
        max_interval_RR = 0
        i_RR = 0

        for i_peak, peak in enumerate(ecg_r_peaks):
            if i_peak == n_peaks - 1:
                continue           
            interval_size = ecg_r_peaks[i_peak+1] - ecg_r_peaks[i_peak]
            if interval_size > max_interval_RR:
                max_interval_RR = interval_size
                i_RR = i_peak

        boundies = ecg_r_peaks[i_RR:i_RR+2]
        diag = row["diag"]
        new_row = {
            'db': row['db'],
            'filename': row['filename'],
            'diag': diag,
            'data': row['data'][:, boundies[0]:boundies[1]]
        }
        
        if PRESENTATION_MODE:
            if not os.path.exists(f'../workdata/aioros/{diag}/sample_{i}/'):
                os.makedirs(f'../workdata/aioros/{diag}/sample_{i}/')
            plot = sns.lineplot(
            x=np.arange(0, len(ecg_signal), 1, dtype=int),
            y=ecg_signal)
            plot.axvline(boundies[0], color='red')
            plot.axvline(boundies[1], color='red')
            fig = plot.get_figure()
            fig.set_size_inches(25,15)
            fig.savefig(f"../workdata/aioros/{diag}/sample_{i}/sample_v5.png", facecolor='white', transparent=False, dpi=300) 
            fig.clf()
            for j, lead in enumerate(leads):
                plot = sns.lineplot(x=np.arange(0, boundies[1]-boundies[0], 1, dtype=int), 
                                    y=new_row['data'][j])
                fig = plot.get_figure()
                fig.savefig(f"../workdata/aioros/{diag}/sample_{i}/lead_{lead}.png", facecolor='white', transparent=False, dpi=300) 
                fig.clf()
        df2 = df2.append(new_row, ignore_index=True)
    except Exception as e:
        print(e)
    
df2.to_pickle(f'../workdata/aioros/aioros_large.pkl')

0/9951
1/9951
2/9951
3/9951
4/9951
5/9951
6/9951
7/9951
8/9951
9/9951
10/9951
11/9951
12/9951
13/9951
14/9951
15/9951
16/9951
17/9951
18/9951
19/9951
20/9951
21/9951
22/9951
23/9951
24/9951
25/9951
26/9951
27/9951
28/9951
29/9951
30/9951
31/9951
32/9951
33/9951
34/9951
35/9951
36/9951
37/9951
38/9951
39/9951
40/9951
41/9951
42/9951
43/9951
44/9951
45/9951
46/9951
47/9951
48/9951
49/9951
50/9951
51/9951
52/9951
53/9951
54/9951
55/9951
56/9951
57/9951
58/9951
59/9951
60/9951
61/9951
62/9951
63/9951
64/9951
65/9951
66/9951
67/9951
68/9951
69/9951
70/9951
index 0 is out of bounds for axis 0 with size 0
71/9951
72/9951
73/9951
74/9951
75/9951
76/9951
77/9951
78/9951
79/9951
80/9951
81/9951
82/9951
83/9951
84/9951
85/9951
86/9951
87/9951
88/9951
89/9951
90/9951
91/9951
92/9951
93/9951
94/9951
95/9951
96/9951
97/9951
98/9951
99/9951
100/9951
101/9951
102/9951
103/9951
104/9951
105/9951
106/9951
107/9951
108/9951
index 0 is out of bounds for axis 0 with size 0
109/9951
110/9951
111/9951
112/99

950/9951
951/9951
952/9951
953/9951
954/9951
955/9951
956/9951
957/9951
958/9951
959/9951
960/9951
961/9951
962/9951
963/9951
964/9951
index 0 is out of bounds for axis 0 with size 0
965/9951
966/9951
967/9951
968/9951
969/9951
970/9951
971/9951
972/9951
973/9951
974/9951
975/9951
976/9951
977/9951
978/9951
979/9951
980/9951
981/9951
982/9951
983/9951
984/9951
985/9951
986/9951
987/9951
988/9951
989/9951
990/9951
991/9951
992/9951
993/9951
994/9951
995/9951
996/9951
997/9951
998/9951
999/9951
1000/9951
1001/9951
1002/9951
1003/9951
1004/9951
1005/9951
1006/9951
1007/9951
1008/9951
1009/9951
1010/9951
1011/9951
1012/9951
1013/9951
1014/9951
1015/9951
1016/9951
1017/9951
1018/9951
1019/9951
1020/9951
1021/9951
1022/9951
1023/9951
1024/9951
1025/9951
1026/9951
1027/9951
1028/9951
1029/9951
1030/9951
1031/9951
index 0 is out of bounds for axis 0 with size 0
1032/9951
1033/9951
1034/9951
1035/9951
1036/9951
1037/9951
1038/9951
1039/9951
1040/9951
1041/9951
1042/9951
1043/9951
1044/9951
1045

1757/9951
1758/9951
1759/9951
1760/9951
1761/9951
1762/9951
1763/9951
1764/9951
1765/9951
1766/9951
1767/9951
1768/9951
1769/9951
1770/9951
index 0 is out of bounds for axis 0 with size 0
1771/9951
1772/9951
1773/9951
1774/9951
1775/9951
1776/9951
1777/9951
1778/9951
1779/9951
1780/9951
1781/9951
1782/9951
1783/9951
1784/9951
1785/9951
1786/9951
1787/9951
1788/9951
1789/9951
1790/9951
1791/9951
1792/9951
1793/9951
1794/9951
1795/9951
1796/9951
1797/9951
1798/9951
1799/9951
1800/9951
1801/9951
1802/9951
1803/9951
1804/9951
1805/9951
1806/9951
1807/9951
1808/9951
1809/9951
1810/9951
1811/9951
1812/9951
1813/9951
1814/9951
1815/9951
1816/9951
1817/9951
1818/9951
1819/9951
1820/9951
1821/9951
1822/9951
1823/9951
1824/9951
1825/9951
1826/9951
1827/9951
1828/9951
1829/9951
1830/9951
1831/9951
1832/9951
1833/9951
1834/9951
1835/9951
1836/9951
1837/9951
1838/9951
1839/9951
1840/9951
1841/9951
1842/9951
1843/9951
1844/9951
1845/9951
1846/9951
1847/9951
1848/9951
1849/9951
1850/9951
1851/9951
18

2607/9951
2608/9951
2609/9951
2610/9951
2611/9951
2612/9951
2613/9951
2614/9951
2615/9951
2616/9951
2617/9951
2618/9951
2619/9951
2620/9951
2621/9951
2622/9951
2623/9951
2624/9951
2625/9951
2626/9951
2627/9951
2628/9951
2629/9951
2630/9951
2631/9951
2632/9951
2633/9951
2634/9951
2635/9951
2636/9951
2637/9951
2638/9951
2639/9951
2640/9951
2641/9951
2642/9951
2643/9951
2644/9951
2645/9951
2646/9951
2647/9951
2648/9951
2649/9951
2650/9951
2651/9951
2652/9951
2653/9951
2654/9951
2655/9951
2656/9951
2657/9951
2658/9951
2659/9951
2660/9951
2661/9951
2662/9951
2663/9951
2664/9951
2665/9951
2666/9951
2667/9951
2668/9951
2669/9951
2670/9951
2671/9951
2672/9951
2673/9951
2674/9951
2675/9951
2676/9951
2677/9951
2678/9951
2679/9951
2680/9951
2681/9951
2682/9951
2683/9951
2684/9951
2685/9951
2686/9951
2687/9951
2688/9951
2689/9951
2690/9951
2691/9951
2692/9951
2693/9951
2694/9951
2695/9951
2696/9951
2697/9951
2698/9951
2699/9951
2700/9951
2701/9951
2702/9951
2703/9951
2704/9951
2705/9951
2706/9951


3445/9951
3446/9951
3447/9951
3448/9951
3449/9951
3450/9951
3451/9951
3452/9951
3453/9951
3454/9951
3455/9951
3456/9951
3457/9951
3458/9951
3459/9951
3460/9951
3461/9951
3462/9951
3463/9951
3464/9951
3465/9951
3466/9951
3467/9951
3468/9951
3469/9951
3470/9951
3471/9951
3472/9951
3473/9951
3474/9951
3475/9951
3476/9951
3477/9951
3478/9951
3479/9951
3480/9951
3481/9951
3482/9951
3483/9951
3484/9951
3485/9951
3486/9951
3487/9951
3488/9951
3489/9951
3490/9951
3491/9951
3492/9951
3493/9951
3494/9951
3495/9951
3496/9951
3497/9951
3498/9951
3499/9951
3500/9951
3501/9951
3502/9951
3503/9951
3504/9951
3505/9951
3506/9951
3507/9951
3508/9951
3509/9951
3510/9951
3511/9951
3512/9951
3513/9951
3514/9951
3515/9951
3516/9951
3517/9951
3518/9951
3519/9951
3520/9951
3521/9951
3522/9951
3523/9951
3524/9951
3525/9951
3526/9951
3527/9951
3528/9951
3529/9951
3530/9951
3531/9951
3532/9951
3533/9951
3534/9951
3535/9951
3536/9951
3537/9951
3538/9951
3539/9951
3540/9951
3541/9951
3542/9951
3543/9951
3544/9951


...

5967/9951
5968/9951
5969/9951
5970/9951
5971/9951
5972/9951
5973/9951
5974/9951
5975/9951
5976/9951
5977/9951
5978/9951
5979/9951
5980/9951
5981/9951
5982/9951
5983/9951
5984/9951
5985/9951
5986/9951
5987/9951
5988/9951
5989/9951
5990/9951
5991/9951
5992/9951
5993/9951
5994/9951
5995/9951
5996/9951
5997/9951
5998/9951
5999/9951
6000/9951
6001/9951
6002/9951
6003/9951
6004/9951
6005/9951
6006/9951
6007/9951
6008/9951
6009/9951
6010/9951
6011/9951
6012/9951
6013/9951
6014/9951
6015/9951
6016/9951
6017/9951
6018/9951
6019/9951
6020/9951
6021/9951
6022/9951
6023/9951
6024/9951
6025/9951
6026/9951
6027/9951
6028/9951
6029/9951
6030/9951
6031/9951
6032/9951
6033/9951
6034/9951
6035/9951
6036/9951
6037/9951
6038/9951
6039/9951
6040/9951
6041/9951
6042/9951
6043/9951
6044/9951
6045/9951
6046/9951
6047/9951
6048/9951
6049/9951
6050/9951
6051/9951
6052/9951
6053/9951
6054/9951
6055/9951
6056/9951
6057/9951
6058/9951
6059/9951
6060/9951
6061/9951
6062/9951
6063/9951
6064/9951
6065/9951
6066/9951


6775/9951
6776/9951
6777/9951
6778/9951
6779/9951
6780/9951
6781/9951
6782/9951
6783/9951
6784/9951
6785/9951
6786/9951
6787/9951
6788/9951
6789/9951
6790/9951
6791/9951
6792/9951
6793/9951
6794/9951
6795/9951
6796/9951
6797/9951
6798/9951
6799/9951
6800/9951
6801/9951
6802/9951
6803/9951
6804/9951
6805/9951
6806/9951
6807/9951
6808/9951
6809/9951
6810/9951
6811/9951
6812/9951
6813/9951
6814/9951
6815/9951
6816/9951
6817/9951
6818/9951
6819/9951
6820/9951
6821/9951
6822/9951
6823/9951
6824/9951
6825/9951
6826/9951
6827/9951
6828/9951
6829/9951
6830/9951
6831/9951
6832/9951
6833/9951
6834/9951
6835/9951
6836/9951
6837/9951
6838/9951
6839/9951
6840/9951
6841/9951
6842/9951
6843/9951
6844/9951
6845/9951
6846/9951
6847/9951
6848/9951
6849/9951
6850/9951
6851/9951
6852/9951
6853/9951
6854/9951
6855/9951
6856/9951
6857/9951
6858/9951
6859/9951
6860/9951
6861/9951
6862/9951
6863/9951
6864/9951
6865/9951
6866/9951
6867/9951
6868/9951
6869/9951
6870/9951
6871/9951
6872/9951
6873/9951
6874/9951


7597/9951
7598/9951
7599/9951
7600/9951
7601/9951
7602/9951
7603/9951
7604/9951
7605/9951
7606/9951
7607/9951
7608/9951
7609/9951
7610/9951
7611/9951
7612/9951
7613/9951
7614/9951
7615/9951
7616/9951
7617/9951
7618/9951
7619/9951
7620/9951
7621/9951
7622/9951
7623/9951
7624/9951
7625/9951
7626/9951
7627/9951
7628/9951
7629/9951
7630/9951
7631/9951
7632/9951
7633/9951
7634/9951
7635/9951
7636/9951
7637/9951
7638/9951
7639/9951
7640/9951
7641/9951
7642/9951
7643/9951
7644/9951
7645/9951
7646/9951
7647/9951
7648/9951
7649/9951
7650/9951
7651/9951
7652/9951
7653/9951
7654/9951
7655/9951
7656/9951
7657/9951
7658/9951
7659/9951
7660/9951
7661/9951
7662/9951
7663/9951
7664/9951
7665/9951
7666/9951
7667/9951
7668/9951
7669/9951
7670/9951
7671/9951
7672/9951
7673/9951
7674/9951
7675/9951
7676/9951
7677/9951
7678/9951
7679/9951
7680/9951
7681/9951
7682/9951
7683/9951
7684/9951
7685/9951
7686/9951
7687/9951
7688/9951
7689/9951
7690/9951
7691/9951
7692/9951
7693/9951
7694/9951
7695/9951
7696/9951


8465/9951
8466/9951
8467/9951
8468/9951
8469/9951
8470/9951
8471/9951
8472/9951
8473/9951
8474/9951
8475/9951
8476/9951
8477/9951
8478/9951
8479/9951
8480/9951
8481/9951
8482/9951
8483/9951
8484/9951
8485/9951
8486/9951
8487/9951
8488/9951
8489/9951
8490/9951
8491/9951
8492/9951
8493/9951
8494/9951
8495/9951
8496/9951
8497/9951
8498/9951
8499/9951
8500/9951
8501/9951
8502/9951
8503/9951
8504/9951
8505/9951
8506/9951
8507/9951
8508/9951
8509/9951
8510/9951
8511/9951
8512/9951
8513/9951
8514/9951
8515/9951
8516/9951
8517/9951
8518/9951
8519/9951
8520/9951
8521/9951
8522/9951
8523/9951
8524/9951
8525/9951
8526/9951
8527/9951
8528/9951
8529/9951
8530/9951
8531/9951
8532/9951
8533/9951
8534/9951
8535/9951
8536/9951
8537/9951
8538/9951
8539/9951
8540/9951
8541/9951
8542/9951
8543/9951
8544/9951
8545/9951
8546/9951
8547/9951
8548/9951
8549/9951
8550/9951
8551/9951
8552/9951
8553/9951
8554/9951
8555/9951
8556/9951
8557/9951
8558/9951
8559/9951
8560/9951
8561/9951
8562/9951
8563/9951
8564/9951


9302/9951
9303/9951
9304/9951
9305/9951
9306/9951
9307/9951
9308/9951
9309/9951
9310/9951
9311/9951
9312/9951
9313/9951
9314/9951
9315/9951
9316/9951
9317/9951
9318/9951
9319/9951
9320/9951
9321/9951
9322/9951
9323/9951
9324/9951
9325/9951
9326/9951
9327/9951
9328/9951
9329/9951
9330/9951
9331/9951
9332/9951
9333/9951
9334/9951
9335/9951
9336/9951
9337/9951
9338/9951
9339/9951
9340/9951
9341/9951
9342/9951
9343/9951
9344/9951
9345/9951
9346/9951
9347/9951
9348/9951
9349/9951
9350/9951
9351/9951
9352/9951
9353/9951
9354/9951
9355/9951
9356/9951
9357/9951
9358/9951
9359/9951
9360/9951
9361/9951
9362/9951
9363/9951
9364/9951
9365/9951
9366/9951
9367/9951
9368/9951
9369/9951
9370/9951
9371/9951
9372/9951
9373/9951
9374/9951
9375/9951
9376/9951
9377/9951
9378/9951
9379/9951
9380/9951
9381/9951
9382/9951
9383/9951
9384/9951
9385/9951
9386/9951
9387/9951
9388/9951
9389/9951
9390/9951
9391/9951
9392/9951
9393/9951
9394/9951
9395/9951
9396/9951
9397/9951
9398/9951
9399/9951
9400/9951
9401/9951
